# 1. Exploratory data analysis

In exploratory data analysis we want to **understand and trust** our data.

We will use an example a dataset with salaries from a fictitious company. This dataset comes from $\color{red}{\text{TODO: ADD SOURCE HERE}}$.

Our objective is to check if there are signs of gender discrimination in the salaries.

In this part we explore the dataset in two steps: load the dataset and understand it.

## Step 1 Load the dataset

The dataset is stored in a CSV file.

Python has a powerful library for data management and analyis: [Pandas](https://pandas.pydata.org/). We will use from the very beginning, to load the dataset.

In [1]:
import pandas as pd

salaries = pd.read_csv('data/salaries.csv')

We now have a [Pandas `DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) with the dataset.

In [2]:
print(type(salaries))

<class 'pandas.core.frame.DataFrame'>


## Step 2 Understand the data

In this step we want to answer a few basic questions about the dataset:

1. How many samples does it have?
1. What pieces of information does it have and in what format are they stored?
1. Does it have incomplete samples (rows missing data)?
1. Does each sample make sense, i.e. is consistent with what it is supposed to represent?
1. Do we need to transform any piece of data to a more convenient or useful value and format?

#### 1. How many samples does it have?

In [3]:
rows, columns = salaries.shape
print('The dataset has {:,} rows and {} columns'.format(rows, columns))

The dataset has 1,192 rows and 6 columns


#### 2. What pieces of information does it have and in what format are they stored?

What pieces of information we have:

In [4]:
print(salaries.columns)

Index(['earn', 'height', 'sex', 'ed', 'age', 'race'], dtype='object')


In [5]:
display(salaries.head())

,earn,height,sex,ed,age,race
0,50000.0,74.424439,male,16,45,white
1,60000.0,65.537543,female,16,58,white
2,30000.0,63.629198,female,16,29,white
3,50000.0,63.108562,female,16,91,other
4,51000.0,63.402484,female,17,39,white


Most datasets come with a description of each column. In the absence of that information, we can infer what the columns represent from their names and their values.

- `earn` is the annual salary.
- `height` is the height in inches, but the number of digits indicates that it was converted from the metric system. We will adjust this later.
- `sex` is a categorical value. We will check this later.
- `ed` is the number of education years, although it's unclear how these years were counted.
- `age` is the complete number of years of age.
- `race` is a categorical value. We will also check this later.

The format of each column:

In [6]:
print(salaries.dtypes)

earn      float64
height    float64
sex        object
ed          int64
age         int64
race       object
dtype: object


The format seems to match the pieces of information. All numerical values are already stored as numbers (`float64`) and all categorical values are stored as string (`object`).

#### 3. Does it have incomplete samples (rows missing data)?

The definition of _incomplete_ varies by data type:

- Floating point numbers: represented with the special value [NumPy.NaN](https://numpy.org/devdocs/reference/constants.html#numpy.NAN).
- Integers and strings: they don't have a special value to represent missing values. They may be enconded with the Python constant [None](https://docs.python.org/3/library/constants.html#None), or with a value out of the expected range, e.g. `-1` for a missing integer value or a zero-length string (`''`) for strings.

Let's start by checking if any column has `NaN` or `None`.

In [7]:
print(salaries.isnull().sum())

earn      0
height    0
sex       0
ed        0
age       0
race      0
dtype: int64


Breaking this down in steps: first we use `isnull)` to check if there is any value, anywhere that is set to `NaN` or `None`. This returns a matrix of `True` (is `NaN` or `None`) or `False` (has a value).

In [8]:
print(salaries.isnull())

       earn  height    sex     ed    age   race
0     False   False  False  False  False  False
1     False   False  False  False  False  False
2     False   False  False  False  False  False
3     False   False  False  False  False  False
4     False   False  False  False  False  False
...     ...     ...    ...    ...    ...    ...
1187  False   False  False  False  False  False
1188  False   False  False  False  False  False
1189  False   False  False  False  False  False
1190  False   False  False  False  False  False
1191  False   False  False  False  False  False

[1192 rows x 6 columns]


Because `False` is represented as zero and `True` is represented as one, we do not need to check every cell of the matrix. We can add them up (by column) and check if any of them is greater than zero.

In [9]:
print(salaries.isnull().sum())

earn      0
height    0
sex       0
ed        0
age       0
race      0
dtype: int64


This shows that the dataset does not have incomplete data flagged as `NaN` or `None`. We still need to check if values out fof range were used to represent missing integers (e.g. `-1` to indicate unknown age or education level) and if zero-length strings were used to represent missing categorical data (e.g. `''` to indicate unknown sex or rage). We will cover these when we analyze the individual columns later.

At this point we have reasonable confidence that we have a complete dataset. It is enough for a quick check at the stage we are in (exploratory data analysys). To understand more about missing data and how to deal with them, see this Pandas [Pandas comprehensive article on intricacies of missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html).

#### 4. Does each sample make sense, i.e. is consistent with what it is supposed to represent?

The next step is to check if the samples are consistent with the data they are supposed to represent. 

We are looking for values that do not make sense, for example someone with 150 years of age, or one year of education, or race equals to 'United States'. These inconsistencies can trickle in at the different stages of data collection: filling out the wrong value in a form, or incorrectly coding a value.

We will start with an overview of the numerical values using [`describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html). It shows the summary statistics for all columns that are either a float or an integer. We will review the categorical columns (`sex` and `race`) later.

In [10]:
salaries.describe()

,earn,height,ed,age
count,1192.000000,1192.000000,1192.000000,1192.000000
mean,23154.773490,66.915154,13.504195,41.378356
std,19472.296925,3.853968,2.420175,15.867428
min,200.000000,57.503219,3.000000,18.000000
25%,10000.000000,64.009746,12.000000,29.000000
50%,20000.000000,66.451265,13.000000,38.000000
75%,30000.000000,69.848100,16.000000,51.000000
max,200000.000000,77.051282,18.000000,91.000000


From the summary statistics, we can observe a few things:

- `earn`: the minimum salary seems low for annual salaries. It may be an error, or it may indicate the column is not _contracted salary_, but perhaps _paid salary_ (someone joined the company recently, or stayed with the company for a short time).
- `height`: seems consistent with an adult population.
- `ed`: the minimum value (three years of education) may indicate errors in those samples.
- `age`: the maximum value is high, although not unrealistic.

Some of the observations indicate that we may need to clean up the dataset before using it. We will do that in a later section.

Now we will check the columns with categorial data, `sex` and `race`. As part of that, we will also complete the incosistency check started in the previous section.

Starting with the `sex` column: let's check the unique values used for it.

In [11]:
print(salaries['sex'].unique())

['male' 'female']


This seems correct and also shows it does not have incomplete data (zero-length strings).

The same for the `race` column.

In [12]:
print(salaries['race'].unique())

['white' 'other' 'hispanic' 'black']


It also looks reasonable for the data it represents and does not have incomplete data.

#### 5. Do we need to transform any piece of data to a more convenient or useful value and format?

The `height` column has an unusual format, with a very large precision. It looks like it was converted from the metric system. We will change it to a more natural format by rounding to one decimal digit.

In [13]:
salaries.height = salaries.height.round(1)

$\color{red}{\text{NOTE TO OGE}}$: I switched the syntax to refer to the column on purpose. I won't spend time explaining it, just wante to show it.

In [14]:
display(salaries.head())

,earn,height,sex,ed,age,race
0,50000.0,74.4,male,16,45,white
1,60000.0,65.5,female,16,58,white
2,30000.0,63.6,female,16,29,white
3,50000.0,63.1,female,16,91,other
4,51000.0,63.4,female,17,39,white


A quick check to make sure we did not make any silly mistake in the conversion.

In [15]:
print(salaries.height.describe())

count    1192.000000
mean       66.914597
std         3.853698
min        57.500000
25%        64.000000
50%        66.450000
75%        69.825000
max        77.100000
Name: height, dtype: float64


It still matches the values we saw in the previoous section.